In [16]:
import json
import random
import pathFile
import datasetPreprocessing as pp

# Make random datasets for training, validation and testing

In [17]:
# Parsed arend data
arend_data = pp.parse_concatenated_json(pathFile.dataset_maker_arend_data)

# Paths for the new dataset
path_train = pathFile.dataset_maker_train
path_valid = pathFile.dataset_maker_valid
path_test = pathFile.dataset_maker_test

# Sizes for random sampling for the new dataset
train_size = 30_000
valid_size = 1_000
test_size = 1_000



train_sample = random.sample(arend_data, train_size)
with open(path_train, 'w', encoding='utf-8') as f:
    for entry in train_sample:
        f.write(json.dumps(pp.simplify_jsonl(entry)) + "\n")

valid_sample = random.sample(arend_data, valid_size)
with open(path_valid, 'w', encoding='utf-8') as f:
    for entry in valid_sample:
        f.write(json.dumps(pp.simplify_jsonl(entry)) + "\n")

test_sample = random.sample(arend_data, test_size)
with open(path_test, 'w', encoding='utf-8') as f:
    for entry in test_sample:
        f.write(json.dumps(pp.simplify_jsonl(entry)) + "\n")